# US Accident

## Import

In [1]:
import seaborn as sns;
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as TTS
from sklearn.model_selection import cross_val_score;
from sklearn.preprocessing import LabelEncoder;
from sklearn.ensemble import RandomForestClassifier

# pipeline elements
from sklearn.decomposition import PCA # PCA = Principal Component Analysis
from sklearn.neighbors import KNeighborsClassifier as KNN 

# pipeline materiaux
from sklearn.pipeline import Pipeline # PCA = Principal Component Analysis
from sklearn.model_selection import GridSearchCV

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score
from itertools import cycle


## Collecte de données

Nous allons étudier un dataset qui regroupent un enssemble d'accidents de voitures ayant eu lieu aux états-unis, soit dans 49 états. Les données ont été récoltées entre 2016 et 2020. Initialement, le dataset faisait environ 3500000 lignes mais étant données sa taille, il a été décider de la réduire afin de ne pas réduire les performances de calculs de nos algorithmes. 

In [13]:
accident = pd.read_csv('data/US_Accidents_June20_mini.csv')

In [14]:
accident.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             100000 non-null  int64  
 1   ID                     100000 non-null  object 
 2   Source                 100000 non-null  object 
 3   TMC                    70447 non-null   float64
 4   Severity               100000 non-null  int64  
 5   Start_Time             100000 non-null  object 
 6   End_Time               100000 non-null  object 
 7   Start_Lat              100000 non-null  float64
 8   Start_Lng              100000 non-null  float64
 9   End_Lat                29553 non-null   float64
 10  End_Lng                29553 non-null   float64
 11  Distance(mi)           100000 non-null  float64
 12  Description            100000 non-null  object 
 13  Number                 35451 non-null   float64
 14  Street                 100000 non-nul

In [15]:
accident.head()

,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,241239,A-241241,MapQuest,201.0,3,2016-11-10 08:51:55,2016-11-10 09:50:56,38.854820,-77.219879,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
1,2727709,A-2727763,Bing,NaN,3,2020-05-29 06:23:07,2020-05-29 06:53:07,30.284830,-90.401800,30.28483,...,False,False,False,False,False,False,Day,Day,Day,Day
2,1709912,A-1709945,MapQuest,241.0,2,2018-08-13 17:53:14,2018-08-13 18:53:05,47.282799,-122.208061,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
3,1730198,A-1730232,MapQuest,241.0,3,2018-08-23 06:11:49,2018-08-23 06:41:28,27.791718,-82.356926,NaN,...,False,False,False,False,False,False,Night,Night,Day,Day
4,3367353,A-3367516,Bing,NaN,3,2018-01-10 17:40:26,2018-01-10 23:40:26,38.514270,-90.375940,38.51191,...,False,False,False,False,False,False,Night,Night,Day,Day


,Unnamed: 0,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,1901733,A-1901773,MapQuest,201.0,3,2018-05-20 12:33:29,2018-05-20 13:03:01,30.266512,-82.753860,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
1,2923127,A-2923261,Bing,NaN,3,2020-04-19 01:54:45,2020-04-19 02:24:45,38.367280,-90.384540,38.367280,...,False,False,False,False,False,False,Night,Night,Night,Night
2,323307,A-323310,MapQuest,201.0,2,2017-02-08 12:29:23,2017-02-08 12:59:01,34.000233,-84.067162,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
3,820712,A-820738,MapQuest,201.0,2,2019-10-15 09:28:51,2019-10-15 10:28:40,35.895634,-78.616295,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
4,2084795,A-2084835,MapQuest,201.0,2,2018-02-18 11:16:29,2018-02-18 11:45:23,36.679970,-78.176910,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2504352,A-2504393,Bing,NaN,3,2016-08-05 18:11:46,2016-08-06 00:11:46,32.839880,-97.329080,32.839450,...,False,False,False,False,False,False,Day,Day,Day,Day
99996,1739407,A-1739441,MapQuest,201.0,2,2018-08-28 20:58:12,2018-08-28 21:27:57,45.976875,-94.408951,NaN,...,False,False,False,False,False,False,Night,Night,Day,Day
99997,2321967,A-2322007,MapQuest,201.0,2,2017-10-19 14:21:04,2017-10-19 15:05:49,30.217627,-97.683296,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
99998,2662444,A-2662492,Bing,NaN,4,2017-04-04 10:11:40,2017-04-04 16:11:40,33.270524,-83.985065,33.277915,...,False,False,False,False,False,False,Day,Day,Day,Day


## Feature Engineering

In [ ]:
X = X.drop(['purchase_time'],axis=1)